<a href="https://colab.research.google.com/github/gabrielhenriquess/TelecomX/blob/main/Analise_TelecomX.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Análise de Evasão churn de Clientes - TelecomX


#📌 Extracão

In [28]:

import pandas as pd
import numpy as np
import requests
import json
import plotly.express as px
import plotly.graph_objects as go
from plotly.subplots import make_subplots
import plotly.io as pio

# Configuração de tema padrão para os gráficos
pio.templates.default = "plotly_white"


In [29]:

# URL dos dados JSON
fonte_dados = 'https://raw.githubusercontent.com/ingridcristh/challenge2-data-science/refs/heads/main/TelecomX_Data.json'

# Requisição HTTP para pegar os dados
resposta = requests.get(fonte_dados)
resposta.raise_for_status()  # Verificar possíveis erros de requisição

# Convertendo o JSON em dicionário Python
dados_cliente = json.loads(resposta.text)

# Normalização das colunas aninhadas
df_clientes = pd.json_normalize(dados_cliente, sep='_')

# Visão geral do dataframe
df_clientes.info()


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 7267 entries, 0 to 7266
Data columns (total 21 columns):
 #   Column                     Non-Null Count  Dtype  
---  ------                     --------------  -----  
 0   customerID                 7267 non-null   object 
 1   Churn                      7267 non-null   object 
 2   customer_gender            7267 non-null   object 
 3   customer_SeniorCitizen     7267 non-null   int64  
 4   customer_Partner           7267 non-null   object 
 5   customer_Dependents        7267 non-null   object 
 6   customer_tenure            7267 non-null   int64  
 7   phone_PhoneService         7267 non-null   object 
 8   phone_MultipleLines        7267 non-null   object 
 9   internet_InternetService   7267 non-null   object 
 10  internet_OnlineSecurity    7267 non-null   object 
 11  internet_OnlineBackup      7267 non-null   object 
 12  internet_DeviceProtection  7267 non-null   object 
 13  internet_TechSupport       7267 non-null   objec

In [30]:

# Checagem de valores únicos por coluna
for atributo in df_clientes.columns:
    print(f"Coluna: {atributo} - Quantidade de valores únicos: {df_clientes[atributo].nunique()}")
    if df_clientes[atributo].nunique() < 50:
        print(df_clientes[atributo].unique())
        print('-' * 40)


Coluna: customerID - Quantidade de valores únicos: 7267
Coluna: Churn - Quantidade de valores únicos: 3
['No' 'Yes' '']
----------------------------------------
Coluna: customer_gender - Quantidade de valores únicos: 2
['Female' 'Male']
----------------------------------------
Coluna: customer_SeniorCitizen - Quantidade de valores únicos: 2
[0 1]
----------------------------------------
Coluna: customer_Partner - Quantidade de valores únicos: 2
['Yes' 'No']
----------------------------------------
Coluna: customer_Dependents - Quantidade de valores únicos: 2
['Yes' 'No']
----------------------------------------
Coluna: customer_tenure - Quantidade de valores únicos: 73
Coluna: phone_PhoneService - Quantidade de valores únicos: 2
['Yes' 'No']
----------------------------------------
Coluna: phone_MultipleLines - Quantidade de valores únicos: 3
['No' 'Yes' 'No phone service']
----------------------------------------
Coluna: internet_InternetService - Quantidade de valores únicos: 3
['DSL

In [31]:

# Verificando linhas duplicadas
total_duplicados = df_clientes.duplicated().sum()
print(f"Total de duplicados: {total_duplicados}")

# Analisando nulos
print(df_clientes.isnull().sum())

# Verificando campos vazios (strings em branco)
vazios = df_clientes.apply(lambda x: x.astype(str).str.strip() == '').sum()
print(vazios[vazios > 0])


Total de duplicados: 0
customerID                   0
Churn                        0
customer_gender              0
customer_SeniorCitizen       0
customer_Partner             0
customer_Dependents          0
customer_tenure              0
phone_PhoneService           0
phone_MultipleLines          0
internet_InternetService     0
internet_OnlineSecurity      0
internet_OnlineBackup        0
internet_DeviceProtection    0
internet_TechSupport         0
internet_StreamingTV         0
internet_StreamingMovies     0
account_Contract             0
account_PaperlessBilling     0
account_PaymentMethod        0
account_Charges_Monthly      0
account_Charges_Total        0
dtype: int64
Churn                    224
account_Charges_Total     11
dtype: int64


#🔧 Transformação

In [32]:

# Substituindo valores vazios na coluna de cobranças totais
df_clientes['account_Charges_Total'] = df_clientes['account_Charges_Total'].replace(' ', 0).astype(np.float64)

# Convertendo colunas textuais para minúsculo
colunas_texto = df_clientes.select_dtypes(include='object').columns.drop('customerID')
for coluna in colunas_texto:
    df_clientes[coluna] = df_clientes[coluna].str.lower()

# Eliminando registros com evasao em branco
df_clientes = df_clientes[df_clientes['Churn'].str.strip() != '']
print(f"Total de registros após limpeza: {len(df_clientes)}")


Total de registros após limpeza: 7043


In [33]:

# Novo nome das colunas
mapeamento_colunas = {
    'customerID': 'id_cliente',
    'Churn': 'evasao',
    'customer_gender': 'sexo',
    'customer_SeniorCitizen': 'categoria_idoso',
    'customer_Partner': 'tem_parceiro',
    'customer_Dependents': 'tem_dependentes',
    'customer_tenure': 'tempo_contrato',
    'phone_PhoneService': 'telefone_ativo',
    'phone_MultipleLines': 'multiplas_linhas',
    'internet_InternetService': 'tipo_internet',
    'internet_OnlineSecurity': 'seguranca_online',
    'internet_OnlineBackup': 'backup_nuvem',
    'internet_DeviceProtection': 'protecao_aparelho',
    'internet_TechSupport': 'suporte_online',
    'internet_StreamingTV': 'tv_stream',
    'internet_StreamingMovies': 'filmes_stream',
    'account_Contract': 'modelo_contrato',
    'account_PaperlessBilling': 'fatura_online',
    'account_PaymentMethod': 'pagamento',
    'account_Charges_Monthly': 'mensalidade',
    'account_Charges_Total': 'total_gasto'
}

df_clientes = df_clientes.rename(columns=mapeamento_colunas)


In [34]:

# Dicionário de tradução de valores
traducao_valores = {
    'female': 'mulher',
    'male': 'homem',
    'yes': 'sim',
    'no': 'nao',
    'fiber optic': 'fibra',
    'no phone service': 'sem_telefone',
    'no internet service': 'sem_internet',
    'one year': 'anual',
    'month-to-month': 'mensal',
    'two year': 'bienal',
    'mailed check': 'cheque_correio',
    'electronic check': 'cheque_online',
    'credit card (automatic)': 'cartao_auto',
    'bank transfer (automatic)': 'transferencia_auto'
}

# Aplicando tradução em todas as colunas de texto
for col in df_clientes.columns:
    if df_clientes[col].dtype == 'object':
        df_clientes[col] = df_clientes[col].replace(traducao_valores)


In [35]:

# Nova coluna: valor diário
df_clientes['media_diaria'] = ((df_clientes['total_gasto'] / df_clientes['tempo_contrato']) / 30).round(2)
df_clientes['mensalidade'] = df_clientes['mensalidade'].round(2)

# Reposicionando a coluna
colunas = df_clientes.columns.tolist()
colunas.insert(19, colunas.pop(colunas.index('media_diaria')))
df_clientes = df_clientes[colunas]

# Ajustando coluna de idoso
df_clientes['categoria_idoso'] = df_clientes['categoria_idoso'].map({1: 'sim', 0: 'nao'})


#📊 Carga e análise

In [36]:

# Exemplo de gráfico simples
fig = px.histogram(df_clientes,
                   x='sexo',
                   color='evasao',
                   barmode='group',
                   barnorm='percent',
                   title='Distribuição de Evasão por Sexo')
fig.show()
